# Giris
**Özel katmanlar**<br>
Bu bölümde, modellerde oluşturduğumuz katmanların (layer) detaylarına ineceğiz. İki farklı yöntemle katmanlar oluşturacak ve bu katmanların yapılarını inceleyeceğiz.

In [1]:
import tensorflow as tf

In [2]:
print(tf.config.list_physical_devices('GPU')) #  Colab uzerinden her hangi bir GPU kullanmiyorum!

[]


# Katmanlar: yararlı işlemlerin ortak kümeleri


In [3]:
"""
tf.keras.layers paketinde, katmanlar nesnelerdir. Bir katman oluşturmak için, nesneyi oluşturmanız yeterlidir. Çoğu katman ilk argüman olarak çıktı boyutlarının/kanallarının sayısını alır.
"""
layer = tf.keras.layers.Dense(100)
"""
Giriş boyutlarının sayısı çoğu zaman gereksizdir, çünkü katman ilk kullanıldığında çıkarsanabilir, ancak manuel olarak belirtmek isterseniz sağlanabilir, bu da bazı karmaşık modellerde kullanışlıdır.
"""
layer = tf.keras.layers.Dense(10, input_shape=(None, 5)) # input_shape = egitim icin kullanacaginiz verilerin boyutu

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
"""
Katmanların birçok yararlı yöntemi vardır.
Örneğin, bir katmandaki tüm değişkenleri `layer.variables` kullanarak ve eğitilebilir değişkenleri `layer.trainable_variables` kullanarak inceleyebilirsiniz.
Bu durumda tam bağlı bir katman ağırlıklar ve önyargılar için değişkenlere sahip olacaktır.
"""
layer.variables

[<Variable path=dense_1/kernel, shape=(5, 10), dtype=float32, value=[[-0.58841336 -0.1766026   0.00927627 -0.03166872 -0.10236561 -0.21372327
    0.1281628   0.25405425  0.10095936  0.46847957]
  [ 0.1399771  -0.2513245   0.33294922  0.40478784 -0.16126975 -0.44430655
   -0.6167796   0.45537466  0.54277927 -0.323342  ]
  [ 0.3759336   0.4658572  -0.2934001  -0.62758124 -0.01253432 -0.05911058
    0.45803744 -0.10262829  0.5797238   0.0913856 ]
  [ 0.03704572 -0.36706662 -0.4909066  -0.48819298  0.40551406 -0.30271617
    0.11275768 -0.45773733  0.06235129 -0.62630045]
  [ 0.44556886  0.00619912 -0.50637424  0.3110783   0.10558063  0.0301165
    0.26338297  0.134117    0.16470128 -0.45643076]]>,
 <Variable path=dense_1/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>]

In [6]:
layer.kernel, layer.bias

(<Variable path=dense_1/kernel, shape=(5, 10), dtype=float32, value=[[-0.58841336 -0.1766026   0.00927627 -0.03166872 -0.10236561 -0.21372327
    0.1281628   0.25405425  0.10095936  0.46847957]
  [ 0.1399771  -0.2513245   0.33294922  0.40478784 -0.16126975 -0.44430655
   -0.6167796   0.45537466  0.54277927 -0.323342  ]
  [ 0.3759336   0.4658572  -0.2934001  -0.62758124 -0.01253432 -0.05911058
    0.45803744 -0.10262829  0.5797238   0.0913856 ]
  [ 0.03704572 -0.36706662 -0.4909066  -0.48819298  0.40551406 -0.30271617
    0.11275768 -0.45773733  0.06235129 -0.62630045]
  [ 0.44556886  0.00619912 -0.50637424  0.3110783   0.10558063  0.0301165
    0.26338297  0.134117    0.16470128 -0.45643076]]>,
 <Variable path=dense_1/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>)

# Özel katmanların uygulanması
endi katmanınızı uygulamanın en iyi yolu tf.keras.Layer sınıfını genişletip uygulamaktır:<br>
- __init__ , tüm girdiden bağımsız başlatma işlemlerini yapabileceğiniz yerdir
- bulid, giriş tensörlerinin şekillerini bildiğiniz ve başlatmanın geri kalanını yapabileceğiniz yer
- call, ileri hesaplamayı nerede yaparsınız

Değişkenlerinizi oluşturmak için build çağrılana kadar beklemenize gerek olmadığını, bunları __init__ içinde de oluşturabileceğinizi unutmayın. Ancak, bunları build içinde oluşturmanın avantajı, katmanın üzerinde işlem yapacağı girdilerin şekline bağlı olarak sonradan değişken oluşturulmasına olanak sağlamasıdır. Öte yandan, __init__ içinde değişken oluşturmak, değişkenleri oluşturmak için gereken şekillerin açıkça belirtilmesi gerekeceği anlamına gelir.

In [17]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight(name = "kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [18]:
_ = layer(tf.zeros([10, 5]))

In [19]:
print([var.name for var in layer.trainable_variables])

['kernel']


# Modeller: Katmanları oluşturma
Makine öğrenimi modellerinde pek çok ilginç katman benzeri şey, mevcut katmanların birleştirilmesiyle uygulanır. Layers can be nested inside other layers.

Genellikle, Model.fit,Model.evaluate ve Model.save gibi model yöntemlerine ihtiyaç duyduğunuzda keras.Model'den miras alırsınız

Keras.Model'in (keras.layers.Layer yerine) sağladığı bir diğer özellik ise değişkenleri izlemenin yanı sıra, keras.Model'in kendi iç katmanlarını da izlemesi ve böylece bunların incelenmesinin kolaylaşmasıdır.

In [23]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name = '')
    filters1, filters2, filters3 = filters # Katmanlarin boyutlari!

    # Katmanlar yani yapay sinir aglari!
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training = False):
    # Traning(egitim) asmasinda calisan kisim! Yukaridaki katmanlari cagirarak islem yapiyoruz!
    x = self.conv2a(input_tensor)
    x = self.bn2a(x , training = training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training = training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training = training)

    x += input_tensor
    return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])

In [24]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [25]:
block.layers

[<Conv2D name=conv2d_6, built=True>,
 <BatchNormalization name=batch_normalization_6, built=True>,
 <Conv2D name=conv2d_7, built=True>,
 <BatchNormalization name=batch_normalization_7, built=True>,
 <Conv2D name=conv2d_8, built=True>,
 <BatchNormalization name=batch_normalization_8, built=True>]

In [26]:
len(block.variables) # modelin toplam öğrenilebilir parametre sayısını (katman bazında) gösterir.

18

In [27]:
block.summary()

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (1, 2, 3, 1)                │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (1, 2, 3, 1)                │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (1, 2, 3, 2)                │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (1, 2, 3, 2)                │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (1, 2, 3, 3)                │               9 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (1, 2, 3, 3)                │              12 │
│ (BatchNormalization)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)

Ancak çoğu zaman, birçok katmanı oluşturan modeller basitçe bir katmanı diğerinin ardından çağırır. Bu, tf.keras.Sequential kullanılarak çok az kodla yapılabilir:

In [28]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [29]:
my_seq.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, None, None, 1)       │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, None, None, 1)       │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, None, None, 2)       │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, None, None, 2)       │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, None, None, 3)       │               9 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, None, None, 3)       │              12 │
│ (BatchNormalization)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)

# Sonuc
Katmanları kısaca incelediğimiz ve iki farklı yöntemle katman oluşturma tekniklerini öğrendiğimiz bir bölüm oldu. Umarım sizin için faydalı olmuştur. Eğer sorularınız olursa aşağıdaki linklerden bana ulaşabilirsiniz. İyi çalışmalar dilerim!

[Linkedin](https://www.linkedin.com/in/ihsancenkiz/)<br>
[Github](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)